In [1]:
import numpy as np
import torch
from shapely import Polygon
from torch.utils.data import DataLoader
from tqdm import tqdm
from datasets.SlideSeperatedImageDataset import SlideSeperatedImageDataset
from labelers.GroundTruthLabeler import GroundTruthLabeler
from models.resnet import Resnet18BinaryClassifier
from utils import divide
from pathlib import Path


In [2]:
slides_root_dir = "data/whole-slides/gut"
annotations_root_dir = "data/annotations/json"
candidates_dataset_dir = "output/candidates"
model_output_dir = "output/models"

In [3]:
data_split_dict = torch.load(f"{model_output_dir}/data-split.pickle")
model = Resnet18BinaryClassifier(model=torch.load(f"{model_output_dir}/model.pickle"))
train_slides = data_split_dict["train_slides"]
test_slides = data_split_dict["test_slides"]
print("Test slides:", test_slides)

Test slides: {'593433', '593447', '593436', '593437', '522934', '593434', '593448'}


In [4]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
model = model.to(device)
batch_size = 256
test_dataset = SlideSeperatedImageDataset(candidates_dataset_dir, test_slides, with_index=True)
# test_dataset = reduce_dataset(test_dataset, discard_ratio=0)
test_loader = DataLoader(test_dataset,
                         batch_size=batch_size,
                         shuffle=False, )

print(f"Candidates: {len(test_dataset):,}")

Device: cuda:0
Candidates: 36,189


In [5]:

model.eval()
indexes = []
predictions = []
with torch.no_grad():
    for i, (x_test, y_test, index) in enumerate(tqdm(iter(test_loader), desc=f"Testing")):
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        test_logits = model.forward(x_test)
        test_loss = model.loss_function(test_logits, y_test)
        test_preds = model.predict(test_logits)
        indexes.append(index)
        predictions.append(test_preds.squeeze())
indexes = torch.cat(indexes).to("cpu")
predictions = torch.cat(predictions).to("cpu")
predicted_positives = indexes[predictions == 1]


Testing: 100%|██████████| 142/142 [00:51<00:00,  2.78it/s]


In [6]:
predicted_positive_bboxes_by_slide = {}
for item_index in predicted_positives:
    file_path = test_dataset.get_item_file_path(item_index)
    file_name = Path(file_path).stem
    slide, x_min, y_min, width, height = file_name.split("_")
    x_min, y_min, width, height = int(x_min), int(y_min), int(width), int(height)
    if not slide in predicted_positive_bboxes_by_slide:
        predicted_positive_bboxes_by_slide[slide] = []
    predicted_positive_bboxes_by_slide[slide].append((x_min, y_min, width, height))

In [7]:
from itertools import product


def calculate_iou(poly, bbox):
    intersection = poly.intersection(bbox).area
    union = poly.union(bbox).area
    return intersection / union if union > 0 else 0


def calculate_iogt(poly, bbox):
    intersection = poly.intersection(bbox).area
    return intersection / poly.area if poly.area > 0 else 0


def calculate_iopd(poly, bbox):
    intersection = poly.intersection(bbox).area
    return intersection / bbox.area if bbox.area > 0 else 0


def calculate_metrics(confusion_matrix):
    tp, fp, fn = confusion_matrix["TP"], confusion_matrix["FP"], confusion_matrix["FN"]
    precision = divide(tp, (tp + fp))
    recall = divide(tp, (tp + fn))
    f1 = divide(2 * precision * recall, (precision + recall))
    return precision, recall, f1


def calculate_iou_confusion_matrix(ground_truth_polygons, predicted_bboxes, i_threshold=0.5):
    gt_polys = [Polygon(pts).buffer(0) for pts in ground_truth_polygons]
    pred_polys = [Polygon([(x, y), (x + w, y), (x + w, y + h), (x, y + h)]).buffer(0) for x, y, w, h in
                  predicted_bboxes]

    iou_matrix = np.zeros((len(gt_polys), len(pred_polys)))
    igt_matrix = np.zeros((len(gt_polys), len(pred_polys)))
    ipd_matrix = np.zeros((len(gt_polys), len(pred_polys)))

    for i, gt in enumerate(gt_polys):
        for j, pred in enumerate(pred_polys):
            iou_matrix[i, j] = calculate_iou(gt, pred)
            igt_matrix[i, j] = calculate_iogt(gt, pred)
            ipd_matrix[i, j] = calculate_iopd(gt, pred)

    matched_gt = set()
    matched_pred = set()

    for i, j in product(range(len(gt_polys)), range(len(pred_polys))):
        if iou_matrix[i, j] > i_threshold or igt_matrix[i, j] > i_threshold or ipd_matrix[i, j] > i_threshold:
            matched_gt.add(i)
            matched_pred.add(j)

    TP = len(matched_gt)
    FP = len(pred_polys) - len(matched_pred)
    FN = len(gt_polys) - len(matched_gt)

    return {
        "TP": TP,
        "FP": FP,
        "FN": FN
    }


total_confusion_matrix = {
    "TP": 0,
    "FP": 0,
    "FN": 0
}
ground_truth_labeler = GroundTruthLabeler("data/labels/slide-annotations/all.json",
                                          "data/labels/patch-classifications.csv")
for slide_name in test_slides:
    ground_truth_positive_regions = ground_truth_labeler.get_positive_regions(slide_name)
    predicted_positive_bboxes = predicted_positive_bboxes_by_slide.get(slide_name, [])
    confusion_matrix = calculate_iou_confusion_matrix(ground_truth_positive_regions, predicted_positive_bboxes)
    tp, fp, fn = confusion_matrix["TP"], confusion_matrix["FP"], confusion_matrix["FN"]
    precision, recall, f1 = calculate_metrics(confusion_matrix)

    total_confusion_matrix["TP"] += tp
    total_confusion_matrix["FP"] += fp
    total_confusion_matrix["FN"] += fn

    n_ground_truth_pos = len(ground_truth_positive_regions)
    n_cv_candidate_pos = test_dataset.slide_to_dataset[slide_name].labels.sum().item()

    print(
        f"{slide_name}: {n_ground_truth_pos:03d} ground truth positives, {n_cv_candidate_pos:03d} positive candidate patches, precision: {precision:.6f}, recall: {recall:.6f}, f1: {f1:.6f}")
total_precision, total_recall, total_f1 = calculate_metrics(total_confusion_matrix)
print()
print(f"Overall: precision: {total_precision:.6f}, recall: {total_recall:.6f}, f1: {total_f1:.6f}")


593433: 003 ground truth positives, 006 positive candidate patches, precision: 0.046512, recall: 0.666667, f1: 0.086957
593447: 025 ground truth positives, 030 positive candidate patches, precision: 0.000000, recall: 0.000000, f1: 0.000000
593436: 170 ground truth positives, 215 positive candidate patches, precision: 0.629032, recall: 0.688235, f1: 0.657303
593437: 094 ground truth positives, 128 positive candidate patches, precision: 0.468750, recall: 0.159574, f1: 0.238095
522934: 182 ground truth positives, 211 positive candidate patches, precision: 0.200000, recall: 0.104396, f1: 0.137184
593434: 027 ground truth positives, 030 positive candidate patches, precision: 0.250000, recall: 0.444444, f1: 0.320000
593448: 013 ground truth positives, 014 positive candidate patches, precision: 0.181818, recall: 0.153846, f1: 0.166667

Overall: precision: 0.386574, recall: 0.324903, f1: 0.353066
